In [ ]:
! pip install networkx

In [ ]:
! pip install --upgrade pip setuptools wheel

In [ ]:
! pip install pandas

In [ ]:
! pip install matplotlib

In [ ]:
from itertools import combinations
import copy
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import glob
import string
import random
import numpy as np
from collections import OrderedDict

<h3>Generate Graph</h3>

In [ ]:
#vertexes = [1000, 2000, 4000, 8000, 16000, 32000] #[100,200,300,400,500,600]
#vertexes = [n for n in range(10000, 100001, 10000)]
n = 100000

In [ ]:
kvals = [5, 10, 15, 20, 25, 30, 35, 40]
p = 0.2
def small_world_graph():
    for k in kvals: #idx, m in enumerate(mvals):
        g = nx.watts_strogatz_graph(n, k, p)
        edges = g.edges()
        file_name = './synth-graphs/small_world_graph_m_'+str(k)+'.txt'
        with open(file_name, 'w') as fp:
            fp.write('\n'.join('{} {}'.format(x[0],x[1]) for x in edges))

In [ ]:
small_world_graph()

In [ ]:
from networkx.utils import py_random_state

def _random_subset(seq, m, rng):
    """Return m unique elements from seq.

    This differs from random.sample which can return repeated
    elements if seq holds repeated elements.

    Note: rng is a random.Random or numpy.random.RandomState instance.
    """
    targets = set()
    while len(targets) < m:
        x = rng.choice(seq)
        targets.add(x)
    return targets

@py_random_state(2)
def barabasi_albert_graph_ext(n, m, seed=None, initial_graph=None):
    """Returns a random graph using Barabási–Albert preferential attachment

    A graph of $n$ nodes is grown by attaching new nodes each with $m$
    edges that are preferentially attached to existing nodes with high degree.

    Parameters
    ----------
    n : int
        Number of nodes
    m : int
        Number of edges to attach from a new node to existing nodes
    seed : integer, random_state, or None (default)
        Indicator of random number generation state.
        See :ref:`Randomness<randomness>`.
    initial_graph : Graph or None (default)
        Initial network for Barabási–Albert algorithm.
        It should be a connected graph for most use cases.
        A copy of `initial_graph` is used.
        If None, starts from a star graph on (m+1) nodes.

    Returns
    -------
    G : Graph

    Raises
    ------
    NetworkXError
        If `m` does not satisfy ``1 <= m < n``, or
        the initial graph number of nodes m0 does not satisfy ``m <= m0 <= n``.

    References
    ----------
    .. [1] A. L. Barabási and R. Albert "Emergence of scaling in
       random networks", Science 286, pp 509-512, 1999.
    """

    if m < 1 or m >= n:
        raise nx.NetworkXError(
            f"Barabási–Albert network must have m >= 1 and m < n, m = {m}, n = {n}"
        )

    if initial_graph is None:
        # Default initial graph : star graph on (m + 1) nodes
        G = nx.star_graph(m)
    else:
        if len(initial_graph) < m or len(initial_graph) > n:
            raise nx.NetworkXError(
                f"Barabási–Albert initial graph needs between m={m} and n={n} nodes"
            )
        G = initial_graph.copy()

    # List of existing nodes, with nodes repeated once for each adjacent edge
    repeated_nodes = [n for n, d in G.degree() for _ in range(d)]
    # Start adding the other n - m0 nodes.
    source = len(G)
    while source < n:
        m1 = random.randint(1,m) #randomly chose m1 between 1 and m so that the minimum degree = 1; otherwise it becomes m
        # Now choose m unique nodes from the existing nodes
        # Pick uniformly from repeated_nodes (preferential attachment)
        targets = _random_subset(repeated_nodes, m1, seed)
        # Add edges to m nodes from the source.
        G.add_edges_from(zip([source] * m1, targets))
        # Add one node to the list for each new edge just created.
        repeated_nodes.extend(targets)
        # And the new node "source" has m edges to add to the list.
        repeated_nodes.extend([source] * m1)

        source += 1
    return G

In [ ]:
mvals = [5,15,25,35]
def scale_free_graph():
    for m in mvals: #idx, m in enumerate(mvals):
        g = barabasi_albert_graph_ext(n, m, None)
        edges = g.edges()
        file_name = './synth-graphs/scale_free_graph_m_'+str(m)+'.txt'
        with open(file_name, 'w') as fp:
            fp.write('\n'.join('{} {}'.format(x[0],x[1]) for x in edges))

In [ ]:
scale_free_graph()

### Generate PCE Compatible (.grh) Input Files

In [ ]:
outdir='./synth-graphs/'
indir='./synth-graphs/'
n

In [ ]:
#convert scale free graphs
for m in mvals:
    file_name = 'scale_free_graph_m_'+str(m)
    os.system('../transgrh.pl < '+ indir+file_name+'.txt' +' > ' + outdir+file_name+'.grh')

In [ ]:
#convert small world graphs
for k in kvals:
    in_file_name = 'small_world_graph_m_'+str(k)
    out_file_name = 'small_world_graph_m_'+str(k)
    os.system('../transgrh.pl < '+ indir+ in_file_name +'.txt' +' > ' + outdir+out_file_name+'.grh')

<h3>Transform graph for ODES</h3>

In [ ]:
outdir='./synth-graphs/'
indir='./synth-graphs/'

In [ ]:
text_files = glob.glob(indir + '*.txt')
print(text_files)

In [ ]:
#convert to dict from list of tuples
def convert_dict(li, di): 
    for (a, b) in li:
        di.setdefault(a,[]).append(b) 
        di.setdefault(b,[]).append(a)
    #edges = list(di) 
    di = OrderedDict(sorted(di.items()))
    for (a,b) in di.items():
        di[a] = sorted(b)

    return di 

In [ ]:
for file_name in text_files:
    G = nx.Graph()
    file = open(file_name, "r")
    for edge in file:
        vertex = edge.split()
        G.add_edge(int(vertex[0]), int(vertex[1]))
    file.close()
    edges = list(G.edges())
    edges_dict = {}
    edges_dict = convert_dict(edges, edges_dict)
    #print( edges_dict )
    #break
    file_name = file_name.split("/")
    odes_file = outdir+'odes_'+file_name[2]
    
    file = open(odes_file, "w")
    file.write('{} \n'.format(G.number_of_nodes()))
    for node, edges in edges_dict.items(): 
        edges = str(edges)
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        for char in edges:
           if char in punctuations: 
                edges = edges.replace(char, "")
        file.write('{}\t{} \n'.format(str(node), edges))
    file.close()    